In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard
from datetime import datetime
from typing import Final

import numpy as np
import torch
import torch.nn as nn
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import mean_squared_error, r2_score
from tqdm import tqdm
from torch.utils.tensorboard.writer import SummaryWriter

from tabrel.benchmark.nw_regr import generate_multidim_noisy_data, make_random_r, train_nw_arbitrary, NwModelConfig

class Mlp(nn.Module):
    def __init__(self, 
                 in_dim: int,
                 hidden_dim: int,
                 out_dim: int,
                 dropout: float = .1,
                 ) -> None:
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, out_dim),
        )
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.net(x)

def to_torch(arr: np.ndarray) -> torch.Tensor:
    return torch.tensor(arr, dtype=torch.float32)


def test_mlp() -> None:
    housing = fetch_california_housing()
    x, y = to_torch(housing.data), to_torch(housing.target)
    mlp = Mlp(in_dim=x.shape[1], hidden_dim=64, out_dim=8)
    print(mlp(x).shape)

test_mlp()

In [ ]:
class RelNwMlpRegr(nn.Module):
    """
    Relationship-aware Nadaraya-Watson kernel regression
    """

    def __init__(self, 
                 init_sigma: float,
                 init_r_scale: float,
                 dropout: float,
                 input_dim: int,
                 mlp_hidden_dim: int = 64,
                 mlp_out_dim: int = 8,
                 ) -> None:
        super().__init__()
        self.sigma = nn.Parameter(torch.tensor([float(init_sigma)]))
        self.r_scale = nn.Parameter(torch.tensor([float(init_r_scale)]))
        self.x_tranform = Mlp(
            in_dim=input_dim,
            hidden_dim=mlp_hidden_dim,
            out_dim=mlp_out_dim,
            dropout=dropout)


    def forward(
        self,
        x_backgnd: torch.Tensor,  # (n_backgnd, n_features)
        y_backgnd: torch.Tensor,  # (n_backgnd,)
        x_query: torch.Tensor,  # (n_query, n_features)
        r: torch.Tensor,  # (n_query, n_backgnd)
    ) -> torch.Tensor:
        """
        Returns predicted y: (n_query,)
        """
        x_backgnd, x_query = self.x_tranform(x_backgnd), self.x_tranform(x_query)
        
        n_query, n_backgnd = r.shape
        x_query_exp = x_query.unsqueeze(1).expand(
            -1, n_backgnd, -1
        )  # (n_query, n_backgnd, n_features)
        x_backgnd_exp = x_backgnd.unsqueeze(0).expand(
            n_query, -1, -1
        )  # (n_query, n_backgnd, n_features)

        dists = torch.norm(x_query_exp - x_backgnd_exp, dim=2)

        # Compute kernel weights: (n_query, n_backgnd)
        k_vals = torch.exp(-dists / self.sigma + self.r_scale * r)

        # Normalize weights
        k_sum = k_vals.sum(dim=1, keepdim=True) + 1e-8  # avoid division by zero
        weights = k_vals / k_sum  # (n_query, n_backgnd)

        # Weighted sum of y_backgnd: (n_query,)
        y_pred = torch.matmul(weights, y_backgnd)

        return y_pred


In [ ]:
n_samples: Final[int] = 1000
seed: Final[int] = 42
x_dim: Final[int] = 7
x_np, y_np, c = generate_multidim_noisy_data(n_samples, n_clusters=3, x_dim=x_dim, seed=seed)

n_query = n_back = n_samples // 3
n_train = n_query + n_back
n_val = n_samples - n_train
back_ids = np.arange(n_back)
query_ids = np.arange(n_query) + n_back
train_ids = np.arange(n_train)
val_ids = np.arange(n_val) + n_train

r = make_random_r(seed, c)

n_epochs: Final[int] = 5000
mse, r2, _, _ = train_nw_arbitrary(
    x_backgnd=x_np[back_ids],
    y_backgnd=y_np[back_ids],
    x_query=x_np[query_ids],
    y_query=y_np[query_ids],
    x_val=x_np[val_ids],
    y_val=y_np[val_ids],
    r_query_backgnd=r[query_ids][:, back_ids],
    r_val_nonval=r[val_ids][:, train_ids],
    cfg=NwModelConfig(input_dim=x_dim, trainable_weights_matrix=True,),
    lr=1e-3,
    n_epochs=n_epochs
)

print(mse, r2)

In [ ]:
def train_nw_mlp(
        seed: int,
        mlp_hid_dim: int,
        mlp_out_dim: int,
        dropout: float,
        weight_decay: float,
        writer: SummaryWriter | None) -> float:
    x_torch, y_torch = to_torch(x_np), to_torch(y_np)
    x_train, y_train = x_torch[train_ids], y_torch[train_ids]
    x_mean, x_std = x_train.mean(dim=0), x_train.std(dim=0)
    x_norm = (x_torch - x_mean) / (x_std + 1e-8)

    x_b, y_b = x_norm[back_ids], y_torch[back_ids]
    x_q, y_q = x_norm[query_ids], y_torch[query_ids]
    x_v, y_v = x_norm[val_ids], y_torch[val_ids]

    r_q_b = to_torch(r)[query_ids][:, back_ids]
    r_val_nval = to_torch(r)[val_ids][:, train_ids]

    torch.random.manual_seed(seed)
    model = RelNwMlpRegr(
        init_sigma=1.,
        init_r_scale=1.,
        input_dim=x_dim,
        mlp_hidden_dim=mlp_hid_dim,
        mlp_out_dim=mlp_out_dim,
        dropout=.2,

    )
    optimizer = torch.optim.AdamW(model.parameters(), weight_decay=weight_decay)
    loss_fn = torch.nn.MSELoss()
    last_val_r2: float | None = None
    model.train()
    for epoch in tqdm(range(n_epochs)):
        optimizer.zero_grad()
        y_pred = model(x_b, y_b, x_q, r_q_b)
        loss = loss_fn(y_pred, y_q)
        loss.backward()
        optimizer.step()
        if writer:
            writer.add_scalar("train/loss", loss.item(), epoch)

        if epoch % 20 != 0: continue

        model.eval()
        with torch.no_grad():
            y_v_pred = model(
                x_norm[train_ids],
                y_train,
                x_v,
                r_val_nval,
            )
            y_v_pred_np = y_v_pred.numpy()
            y_v_np = y_v.numpy()

        r2 = r2_score(y_v_np, y_v_pred_np)
        if writer:
            writer.add_scalar("val/mse", mean_squared_error(y_v_np, y_v_pred_np), epoch)
            writer.add_scalar("val/r2", r2, epoch)
        
        last_val_r2 = r2
        model.train()
    
    if last_val_r2 is None:
        raise ValueError("last_val_r2 is None, probably not enough iterations")
    return last_val_r2


In [ ]:
log_dir: Final[str] = "tb_logs" + datetime.isoformat(datetime.now()).replace(":", "_")
print(log_dir)

In [ ]:
%tensorboard --logdir {log_dir}

writer = SummaryWriter(log_dir=log_dir)
train_nw_mlp(
    seed=seed,
    mlp_hid_dim=64,
    mlp_out_dim=6,
    dropout=.2,
    weight_decay=1e-4,
    writer=writer)

In [ ]:
import optuna

def objective(trial: optuna.Trial) -> float:
    weight_decay = trial.suggest_float("weight_decay", 0., 1e-2)
    mlp_hid_dim = trial.suggest_int("mlp_hid_dim", 6, 100)
    mlp_out_dim = trial.suggest_int("mlp_out_dim", 1, 20)
    dropout = trial.suggest_float("dropout", 0., .5)
    return train_nw_mlp(
        seed=seed,
        mlp_hid_dim=mlp_hid_dim,
        mlp_out_dim=mlp_out_dim,
        dropout=dropout,
        weight_decay=weight_decay,
        writer=None,
    )

study = optuna.create_study(direction="maximize", study_name=f"r_deterministic_{datetime.now()}", storage="sqlite:///db.sqlite3")
study.optimize(objective)

code for loading an existing trial

In [ ]:
study = optuna.create_study(
    direction="maximize",
    storage="sqlite:///db.sqlite3",
    study_name="r_deterministic_2025-11-07 14:52:56.356900",
    load_if_exists=True,
)

study.optimize(objective, n_trials=10)
print(f"Best trial: {study.best_trial}")

In [ ]:
# TODO transform Y (with another MLP?)
# TODO another synthetic?